# Hello Triangle

Welcome to your very first tutorial on rendering opengl using pyside2.

What we are going to do now is pretty simple. We shall draw the `Hello Triangle` more specifically a single yellow triangle.
You shall see that once you can draw a triangle, or better a cube, in a graphics api, you would understand the stages of rendering pretty well.

Now run the cell below to see the final version of our triangle

In [1]:
import subprocess

subprocess.run(["python", "./app.py"])

CompletedProcess(args=['python', './app.py'], returncode=0)

Cool eh!

Now let's do a fresh start and create a directory under this one called `myTriangle`.

- Open the terminal
- Go to this directory `cd PATH_TO_GITPROJECT/tutorials/01-triangle/`
- Create a new directory `mkdir myTriangle`
- Inside create to files `touch app.py` and `touch myGLWidget.py`

The `app.py` will hold the window in which the gl widget would live, and `myGLWidget.py` simply defines the OpenGL rendering widget.

Let's start by filling the easy one the `app.py`

### Window Containing the OpenGL Rendering Widget

This is essentially no different from a window containing any other widget, which is one of the reasons why you might want to use `pyside2` in the first place. It gives you to capacity to mix up an opengl widget with other traditional ones.

Here is the list of things we would need:

In [ ]:
from PySide2 import QtWidgets, QtCore, QtGui
from myGLWidget import TriangleGL
from tutorials.utils.window import GLWindow as AppWindow
import sys

Obviously we have not made a TriangleGL widget yet, but we do need the other ones.

The `window.py` contains an application window that contains several widgets alongside of an opengl one. 
We shall see how they would interract with each other in other tutorials. 
For now just remember that resizing the window resizes the opengl widget as well.
Since this is not a tutorial on how to make a qt application we skip the explanation about the code in `window.py`

So our `app.py` looks like the following. 

In [ ]:
from PySide2 import QtWidgets
from myGLWidget import TriangleGL
from tutorials.utils.window import GLWindow as AppWindow
import sys

if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = AppWindow(glwidget=TriangleGL)
    window.show()
    res = app.exec_()
    sys.exit(res)


The last part with `if __name__ == '__main__'` is for dealing with `import` statements.

As you can see it is a fairly simple window which contains 3 sliders and an opengl widget.

Now let's create our OpenGL widget. First, let's see what objects we shall use for the widget

In [ ]:
import numpy as np  # facilitates interfacing with c 
import os  # general path manipulation
import sys  # to send the exit signal if necessary
import ctypes  # a must for communicating with c code under the opengl hood

from PySide2 import QtWidgets, QtCore, QtGui
from PySide2.QtGui import QVector3D  # for attribute/uniform values of type vec3 in shaders
from PySide2.QtGui import QOpenGLVertexArrayObject  # the VAO in opengl jargon
from PySide2.QtGui import QOpenGLBuffer  # a buffer object for storing your data
from PySide2.QtGui import QOpenGLShaderProgram  # the shader program to which we can attach shaders
from PySide2.QtGui import QOpenGLShader  # represents a shader
from PySide2.QtGui import QOpenGLContext  # an opengl context in which a drawing occurs
from PySide2.QtGui import QMatrix4x4  # for attribute/uniform values of type mat4 in shaders
from PySide2.QtGui import QVector4D  # for attribute/uniform values of type vec4 in shaders

from PySide2.QtWidgets import QApplication  # need to display error message 
from PySide2.QtWidgets import QMessageBox  # the box in which the message will appear
from PySide2.QtWidgets import QOpenGLWidget  # the abstract class that we will inherit 
# for constructing our widget

from PySide2.QtCore import QCoreApplication

from PySide2.shiboken2 import VoidPtr  # needed for attribute pointer function

A crucial library for accessing opengl related flags is `pyopengl`. 
So we need to check if it exists if we want to do anything related to opengl in python.

In [ ]:
try:
    from OpenGL import GL as pygl
except ImportError:
    app = QApplication(sys.argv)
    messageBox = QMessageBox(QMessageBox.Critical, "OpenGL hellogl",
                             "PyOpenGL must be installed to run this example.",
                             QMessageBox.Close)
    messageBox.setDetailedText(
        "Run:\npip install PyOpenGL PyOpenGL_accelerate")
    messageBox.exec_()
    sys.exit(1)

Let's see now the actual glwidget constructor.

In [ ]:
class TriangleGL(QOpenGLWidget):
    def __init__(self, parent=None):
        QOpenGLWidget.__init__(self, parent)

        # shaders etc
        triangleTutoDir = os.path.dirname(__file__)
        # triangleTutoDir = os.path.join(triangleTutoDir, os.pardir)
        shaderDir = os.path.join(triangleTutoDir, "shaders")
        availableShaders = ["triangle"]
        self.shaders = {
            name: {
                "fragment": os.path.join(shaderDir, name + ".frag"),
                "vertex": os.path.join(shaderDir, name + ".vert")
            } for name in availableShaders
        }
        self.core = "--coreprofile" in QCoreApplication.arguments()

        # opengl data related
        self.context = QOpenGLContext()
        self.vao = QOpenGLVertexArrayObject()
        self.vbo = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)
        self.program = QOpenGLShaderProgram()

        # some vertex data for corners of triangle
        # please do note that the dimension of the array is 1
        # we shall specify the offset and stride for the
        # vertices of the triangle

        self.vertexData = np.array(
            [-0.5, -0.5, 0.0,  # x, y, z
             0.5, -0.5, 0.0,  # x, y, z
             0.0, 0.5, 0.0],  # x, y, z
            dtype=ctypes.c_float # notice the ctype for interfacing the underlaying c lib
        )
        # triangle color
        self.triangleColor = QVector4D(0.5, 0.5, 0.0, 0.0)  # yellow triangle
        # notice the correspondance the vec4 of fragment shader 
        # and our choice here

It is important to check the state of opengl in our machine. We can do so with the following method. Do not forget that your version of opengl has implication on the functions and the shaders you can use in your code.

In [ ]:
    def getGlInfo(self):
        "Get opengl info"
        info = """
            Vendor: {0}
            Renderer: {1}
            OpenGL Version: {2}
            Shader Version: {3}
            """.format(
            pygl.glGetString(pygl.GL_VENDOR),
            pygl.glGetString(pygl.GL_RENDERER),
            pygl.glGetString(pygl.GL_VERSION),
            pygl.glGetString(pygl.GL_SHADING_LANGUAGE_VERSION)
        )
        return info

Since shaders are a big part of opengl. Let's see the shader related part of our glwidget.

In [ ]:
    def loadShader(self,
                   shaderName: str,
                   shaderType: str):
        "Load shader"
        shader = self.shaders[shaderName]  # we choose the shader from available shaders
        shaderSource = shader[shaderType]  # we take the source of the shader
        if shaderType == "vertex":  # Notice that we specify the type of the shader in the
            shader = QOpenGLShader(QOpenGLShader.Vertex)  # constructor of the qt-shader object 
        else:
            shader = QOpenGLShader(QOpenGLShader.Fragment)
        #
        isCompiled = shader.compileSourceFile(shaderSource) # compilation of the shader
        # we can not attach the shader to program before compilation
        # so it is important check if the compilation occured without error

        if isCompiled is False:
            print(shader.log())
            raise ValueError(
                "{0} shader {2} known as {1} is not compiled".format(
                    shaderType, shaderName, shaderSource
                )
            )
        return shader

    def loadVertexShader(self, shaderName: str):  # loads vertex shader
        "load vertex shader"
        return self.loadShader(shaderName, "vertex")

    def loadFragmentShader(self, shaderName: str):  # loads fragment shader
        "load fragment shader"
        return self.loadShader(shaderName, "fragment")

Now let's see the actual drawing code which corresponds to the drawing loop in an equivalent c/c++ code. 

In [ ]:
 def paintGL(self):  # Notice the uppercase of GL because this paintGL function corresponds to 
        "drawing loop" # a virtual function in c code some you need to implement this function
                       # if you want to draw anything 
        # functions that are available for our current drawing context
        funcs = self.context.functions()

        # clean up what was drawn in the previous frame
        funcs.glClear(pygl.GL_COLOR_BUFFER_BIT)

        # actual drawing code
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)  # we bind the vertex array object
        self.program.bind()  # we bind the program means we activate the program
        funcs.glDrawArrays(pygl.GL_TRIANGLES, # we draw the triangle
                           0,
                           3)
        self.program.release()  # the frame is drawn so we can deactivate the program
        vaoBinder = None  # we can unbind the vao since again the frame is drawn

What if we resize the viewport

In [ ]:
    def resizeGL(self, width: int, height: int):
        "Resize the viewport"
        funcs = self.context.functions()  # get the functions available for our context
        funcs.glViewport(0, 0, width, height)

What if we close the opengl program, we should release the ressources.

In [ ]:
    def cleanUpGl(self):
        "Clean up everything"
        self.context.makeCurrent()  # we first make the context we want to release current
        self.vbo.destroy()  # we destroy the buffer that holds the data
        del self.program  # we delete the shader program, thus free the memory from it
        self.program = None  # we change the value of the pointed reference 
        self.doneCurrent() # we make no context current in the current thread 

Now let's see the most daunting part of the code that is initialization of the gl widget.
First let's see the creation of the context in which the drawing would occur.

In [ ]:
    def initializeGL(self):
        print('gl initial')
        print(self.getGlInfo())
        # create context 
        self.context.create()
        # if the close signal is given we clean up the ressources as per defined above
        self.context.aboutToBeDestroyed.connect(self.cleanUpGl)       

We initialize the function that are available for the current context.

In [ ]:
        # initialize functions
        funcs = self.context.functions()  # we obtain functions for the current context
        funcs.initializeOpenGLFunctions() # we initialize functions
        funcs.glClearColor(1, 1, 1, 1) # the color that will fill the frame when we call the function
        # for cleaning the frame in paintGL

Let's see how we initialize the shaders and shader program.

In [ ]:
        # deal with shaders
        shaderName = "triangle"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        # creating shader program
        self.program = QOpenGLShaderProgram(self.context)
        self.program.addShader(vshader)  # adding vertex shader
        self.program.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program.bindAttributeLocation("aPos", 0) # notice the correspondance of the
        # name aPos in the vertex shader source

        # link shader program
        isLinked = self.program.link()
        print("shader program is linked: ", isLinked)
        # if the program is not linked we won't have any output so
        # it is important to check for it

        # bind the program == activate the program
        self.program.bind()

        # specify uniform value
        colorLoc = self.program.uniformLocation("color") 
        # notice the correspondance of the
        # name color in fragment shader
        # we also obtain the uniform location in order to 
        # set value to it
        self.program.setUniformValue(colorLoc,
                                     self.triangleColor)
        # notice the correspondance of the color type vec4 
        # and the type of triangleColor

Let's create the vao which holds the vertex structure data, and buffer which holds the data itself.

In [ ]:
        # create vao and vbo

        # vao
        isVao = self.vao.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)

        # vbo
        isVbo = self.vbo.create()
        isBound = self.vbo.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

Now let us allocate the space necessary for holding data in buffer.

In [ ]:
        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo.allocate(self.vertexData.tobytes(),  # the actual content of the data
                          floatSize * self.vertexData.size  # the size of the data
                         )

Now let's see how to use vertex array object.

In [ ]:
        funcs.glEnableVertexAttribArray(0)  
        # 0 represent the location of aPos
        # we know this number because it is us who bind it to that location above
        nullptr = VoidPtr(0)  # no idea what we do with this thing.
        funcs.glVertexAttribPointer(0,  # the location of aPos attribute
                                    3,  # 3 for vec3
                                    int(pygl.GL_FLOAT),  # type of value in the coordinates
                                    # notice that we use a flag from opengl
                                    int(pygl.GL_FALSE),  # should we normalize the coordinates
                                    # or not
                                    3 * floatSize, # stride. That is when does the next vertice
                                    # start in the array
                                    nullptr  # offset. From where the coordinates starts
                                    # in the array, since we only have vertex coordinates 
                                    # in the array, we start from 0
                                   )

Now we are done with all the steps. We should release the ressources.

In [ ]:
        self.vbo.release()
        self.program.release()
        vaoBinder = None

Here is all of the method for initializing the glwidget.

In [ ]:
    def initializeGL(self):
        print('gl initial')
        print(self.getGlInfo())
        # create context and make it current
        self.context.create()
        self.context.aboutToBeDestroyed.connect(self.cleanUpGl)
            
        # initialize functions
        funcs = self.context.functions()
        funcs.initializeOpenGLFunctions()
        funcs.glClearColor(1, 1, 1, 1)

        # deal with shaders
        shaderName = "portableTriangle"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        # creating shader program
        self.program = QOpenGLShaderProgram(self.context)
        self.program.addShader(vshader)  # adding vertex shader
        self.program.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program.bindAttributeLocation("aPos", 0)

        # link shader program
        isLinked = self.program.link()
        print("shader program is linked: ", isLinked)

        # bind the program
        self.program.bind()

        # specify uniform value
        colorLoc = self.program.uniformLocation("color")
        self.program.setUniformValue(colorLoc,
                                     self.triangleColor)

        # deal with vao and vbo

        # create vao and vbo

        # vao
        isVao = self.vao.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)

        # vbo
        isVbo = self.vbo.create()
        isBound = self.vbo.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo.allocate(self.vertexData.tobytes(),
                          floatSize * self.vertexData.size)
        funcs.glEnableVertexAttribArray(0)
        nullptr = VoidPtr(0)
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    3 * floatSize,
                                    nullptr)
        self.vbo.release()
        self.program.release()
        vaoBinder = None

Now let's see all of our class which represent the widget.

In [ ]:
class TriangleGL(QOpenGLWidget):
    def __init__(self, parent=None):
        QOpenGLWidget.__init__(self, parent)

        # shaders etc
        projectdir = os.getcwd()
        self.shaders = {
            "portableTriangle": {
                "fragment": """
uniform mediump vec4 color;

void main(void)
{
    gl_FragColor = color;
}""",
                "vertex": """
attribute highp vec3 aPos;
void main(void)
{
    gl_Position = vec4(aPos, 1.0);
}

""",

            }
        }
        self.core = "--coreprofile" in QCoreApplication.arguments()

        # opengl data related
        self.context = QOpenGLContext()
        self.vao = QOpenGLVertexArrayObject()
        self.vbo = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)
        self.program = QOpenGLShaderProgram()

        # some vertex data for corners of triangle
        self.vertexData = np.array(
            [-0.5, -0.5, 0.0,  # x, y, z
             0.5, -0.5, 0.0,  # x, y, z
             0.0, 0.5, 0.0],  # x, y, z
            dtype=ctypes.c_float
        )
        # triangle color
        self.triangleColor = QVector4D(0.5, 0.5, 0.0, 0.0)  # yellow triangle
        # notice the correspondance the vec4 of fragment shader 
        # and our choice here

    def loadShader(self,
                   shaderName: str,
                   shaderType: str):
        "Load shader"
        shader = self.shaders[shaderName]
        shaderSource = shader[shaderType]
        if shaderType == "vertex":
            shader = QOpenGLShader(QOpenGLShader.Vertex)
        else:
            shader = QOpenGLShader(QOpenGLShader.Fragment)
        #
        isCompiled = shader.compileSourceCode(shaderSource)

        if isCompiled is False:
            print(shader.log())
            raise ValueError(
                "{0} shader {2} known as {1} is not compiled".format(
                    shaderType, shaderName, shaderSource
                )
            )
        return shader

    def loadVertexShader(self, shaderName: str):
        "load vertex shader"
        return self.loadShader(shaderName, "vertex")

    def loadFragmentShader(self, shaderName: str):
        "load fragment shader"
        return self.loadShader(shaderName, "fragment")

    def getGlInfo(self):
        "Get opengl info"
        info = """
            Vendor: {0}
            Renderer: {1}
            OpenGL Version: {2}
            Shader Version: {3}
            """.format(
            pygl.glGetString(pygl.GL_VENDOR),
            pygl.glGetString(pygl.GL_RENDERER),
            pygl.glGetString(pygl.GL_VERSION),
            pygl.glGetString(pygl.GL_SHADING_LANGUAGE_VERSION)
        )
        return info

    def initializeGL(self):
        print('gl initial')
        print(self.getGlInfo())
        # create context and make it current
        self.context.create()
        self.context.aboutToBeDestroyed.connect(self.cleanUpGl)
            
        # initialize functions
        funcs = self.context.functions()
        funcs.initializeOpenGLFunctions()
        funcs.glClearColor(1, 1, 1, 1)

        # deal with shaders
        shaderName = "portableTriangle"
        vshader = self.loadVertexShader(shaderName)
        fshader = self.loadFragmentShader(shaderName)

        # creating shader program
        self.program = QOpenGLShaderProgram(self.context)
        self.program.addShader(vshader)  # adding vertex shader
        self.program.addShader(fshader)  # adding fragment shader

        # bind attribute to a location
        self.program.bindAttributeLocation("aPos", 0)

        # link shader program
        isLinked = self.program.link()
        print("shader program is linked: ", isLinked)

        # bind the program
        self.program.bind()

        # specify uniform value
        colorLoc = self.program.uniformLocation("color")
        self.program.setUniformValue(colorLoc,
                                     self.triangleColor)

        # self.useShader("triangle")

        # deal with vao and vbo

        # create vao and vbo

        # vao
        isVao = self.vao.create()
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)

        # vbo
        isVbo = self.vbo.create()
        isBound = self.vbo.bind()

        # check if vao and vbo are created
        print('vao created: ', isVao)
        print('vbo created: ', isVbo)

        floatSize = ctypes.sizeof(ctypes.c_float)

        # allocate space on buffer
        self.vbo.allocate(self.vertexData.tobytes(),
                          floatSize * self.vertexData.size)
        funcs.glEnableVertexAttribArray(0)
        nullptr = VoidPtr(0)
        funcs.glVertexAttribPointer(0,
                                    3,
                                    int(pygl.GL_FLOAT),
                                    int(pygl.GL_FALSE),
                                    3 * floatSize,
                                    nullptr)
        self.vbo.release()
        self.program.release()
        vaoBinder = None

    def cleanUpGl(self):
        "Clean up everything"
        self.context.makeCurrent()
        self.vbo.destroy()
        del self.program
        self.program = None
        self.doneCurrent()

    def resizeGL(self, width: int, height: int):
        "Resize the viewport"
        funcs = self.context.functions()
        funcs.glViewport(0, 0, width, height)

    def paintGL(self):
        "drawing loop"
        funcs = self.context.functions()

        # clean up what was drawn
        funcs.glClear(pygl.GL_COLOR_BUFFER_BIT)

        # actual drawing
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)
        self.program.bind()
        funcs.glDrawArrays(pygl.GL_TRIANGLES,
                           0,
                           3)
        self.program.release()
        vaoBinder = None

All done! If you have come so far, congragulations. Try your triangle by executing the following cell.

In [ ]:
import os
import subprocess
subprocess.run(
    ["python", os.path.join("myTriangle", "app.py")]
)